In [1]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on 
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0   508k      0  0:00:01  0:00:01 --:--:--  508k


In [2]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [4]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [5]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [8]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [9]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [10]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0144, 0.0493, 0.0382,  ..., 0.0377, 0.0198, 0.0276],
        [0.0305, 0.0495, 0.0449,  ..., 0.0339, 0.0387, 0.0169],
        [0.0108, 0.0389, 0.0423,  ..., 0.0185, 0.0476, 0.0135],
        ...,
        [0.0051, 0.0142, 0.0062,  ..., 0.0403, 0.0296, 0.0435],
        [0.0243, 0.0133, 0.0042,  ..., 0.0027, 0.0192, 0.0385],
        [0.0105, 0.0394, 0.0221,  ..., 0.0167, 0.0384, 0.0036]])
item_factors.weight tensor([[0.0460, 0.0045, 0.0159,  ..., 0.0214, 0.0158, 0.0003],
        [0.0497, 0.0310, 0.0243,  ..., 0.0265, 0.0245, 0.0464],
        [0.0429, 0.0170, 0.0299,  ..., 0.0247, 0.0117, 0.0343],
        ...,
        [0.0196, 0.0103, 0.0309,  ..., 0.0397, 0.0339, 0.0094],
        [0.0314, 0.0215, 0.0279,  ..., 0.0313, 0.0440, 0.0136],
        [0.0192, 0.0135, 0.0086,  ..., 0.0356, 0.0132, 0.0427]])


In [11]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()#
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

/home/kangjunekoo/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.065904936209549
iter #1 Loss: 4.745188376020054
iter #2 Loss: 2.4762253165245056
iter #3 Loss: 1.7235483039302875
iter #4 Loss: 1.3473188051112412
iter #5 Loss: 1.1295803306520287
iter #6 Loss: 0.9919814894042039
iter #7 Loss: 0.9004396767483145
iter #8 Loss: 0.8374808989654338
iter #9 Loss: 0.7923246234806661
iter #10 Loss: 0.7596468922072256
iter #11 Loss: 0.7348806808896476
iter #12 Loss: 0.716352135964154
iter #13 Loss: 0.7016081486634796
iter #14 Loss: 0.6904451664783022
iter #15 Loss: 0.6818206596192975
iter #16 Loss: 0.6751752179588764
iter #17 Loss: 0.669780366916947
iter #18 Loss: 0.6658539759174822
iter #19 Loss: 0.6626180183842098
iter #20 Loss: 0.6606972908534979
iter #21 Loss: 0.6588713844037298
iter #22 Loss: 0.6578234883810058
iter #23 Loss: 0.6565826672512263
iter #24 Loss: 0.6558876661631057
iter #25 Loss: 0.6551163215095621
iter #26 Loss: 0.6542609145647378
iter #27 Loss: 0.6534933518561615
iter #28 Loss: 0.6527618819157485
iter #29 Loss: 0.6517409421

In [12]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.4859,  1.3873,  1.0578,  ...,  1.0514,  1.8694,  1.8384],
        [ 1.5357,  0.8904,  1.9075,  ...,  0.0803,  1.1316,  0.5660],
        [ 1.1297, -1.1444,  2.2441,  ...,  0.7498,  0.1134, -1.8530],
        ...,
        [ 0.9857, -0.5377, -0.2009,  ...,  1.8813,  0.9697,  2.2526],
        [ 1.0814,  1.7001,  1.0098,  ...,  0.3897,  0.6825,  1.0839],
        [ 1.0646,  1.2571,  1.6267,  ...,  1.2551,  0.7024,  0.4429]],
       device='cuda:0')
item_factors.weight tensor([[0.3073, 0.6446, 0.4958,  ..., 0.3291, 0.6191, 0.4803],
        [0.8687, 0.6689, 0.2978,  ..., 0.1468, 0.6514, 0.2697],
        [0.3397, 0.4654, 0.5259,  ..., 0.5794, 0.1143, 0.5360],
        ...,
        [0.3459, 0.3360, 0.3581,  ..., 0.3672, 0.3607, 0.3129],
        [0.3805, 0.3709, 0.3772,  ..., 0.3787, 0.3916, 0.3506],
        [0.4006, 0.3935, 0.3900,  ..., 0.4158, 0.3937, 0.4335]],
       device='cuda:0')


In [13]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [14]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [15]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [16]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Jurassic Park (1993)
	 Terminator 2: Judgment Day (1991)
	 Apollo 13 (1995)
	 Fugitive, The (1993)
	 Batman (1989)
	 Aladdin (1992)
	 Lion King, The (1994)
	 Speed (1994)
	 Shrek (2001)
	 Men in Black (a.k.a. MIB) (1997)
Cluster #1
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Braveheart (1995)
	 Fight Club (1999)
	 Toy Story (1995)
	 Seven (a.k.a. Se7en) (1995)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
	 Star Wars: Episode VI - Return of the Jedi (1983)
Cluster #2
	 Mission: Impossible (1996)
	 Batman Forever (1995)
	 Rock, The (1996)
	 Ocean's Eleven (2001)
	 Net, The (1995)
	 Crimson Tide (1995)
	 Firm, The (1993)
	 Broken Arrow (1996)
	 Hangover, The (2009)
	 American President, The (1995)
Cluster #3
	 Batman & Robin (1997)
	 Godzilla (1998)
	 Anaconda (1997)
	 Buffy the Vampire Slayer (1992)
	 Super Mario Bros. (1993)
	 Honey, I Blew Up the Kid (1992)
	 Speed 2: